<h1 style="text-align: center;font-size: 36px;color:darkblue;">Analysis</h1>

# Imports

In [ ]:
import pandas
import numpy
import seaborn
import matplotlib.pyplot as pyplot
import re

# Configurations

In [ ]:
# %matplotlib qt;
%matplotlib inline
pyplot.rcParams["figure.figsize"] = (15, 12)

In [ ]:
def getPaymentMethod(inputString):
    if len(re.findall("^\d{6}",inputString.strip()))>0:
        return "Cheque"
    elif len(re.findall("UPI",inputString.strip()))>0:
        return "UPI"
    elif len(re.findall("^NEFT",inputString.strip()))>0:
        return "NEFT"
    elif len(re.findall("^IMPS",inputString.strip()))>0:
        return "IMPS"
    elif len(re.findall("^ID|^nfs/|^cashnet/",inputString.strip()))>0:
        return "ATM"
    elif len(re.findall("^SMS|^ANNUAL|Keeping|RENEWAL",inputString.strip()))>0:
        return "SMS Charge"
    elif len(re.findall("^ANNUAL",inputString.strip()))>0:
        return "Some ANNUAL Charge"
    elif len(re.findall("^ACH-",inputString.strip()))>0:
        return "Automated Clearing House Payment"
    elif len(re.findall("POS/",inputString.strip()))>0:
        return "Payment done at retail store"
    elif len(re.findall("[\.\/+\:\-\_]",inputString.strip()))==0:
        return "Unknown Type Payment"
    else:
        return "Other"

In [ ]:
def getRecipientName(inputString):
    if len(re.findall("^\d{6}",inputString.strip()))>0:
        return inputString.strip()[7:].strip()
    elif len(re.findall("UPI",inputString.strip()))>0:
        return re.split("[/@]",inputString.strip())[-1].strip()
    elif len(re.findall("^NEFT",inputString.strip()))>0:
        return inputString.strip().split("-")[-1].strip()
    elif len(re.findall("^ACH-",inputString.strip()))>0:
        return inputString.split('-')[1].strip()
    elif len(re.findall("^IMPS",inputString.strip()))>0:
        return inputString.strip().split("/")[2].strip()
    elif len(re.findall("^SMS|^ANNUAL|Keeping|RENEWAL",inputString.strip()))>0:
        return "IDBI Bank"
    elif len(re.findall("^ID|^nfs/|^cashnet/",inputString.strip()))>0:
        return "Self"
    elif len(re.findall("POS/",inputString.strip()))>0:
        return inputString.strip().split("POS/")[-1].strip()
    elif len(re.findall("[\.\/+\:\-\_]",inputString.strip()))==0:
        return inputString.strip()
    else:
        return "Other"

In [ ]:
Statement = pandas.read_csv("../../Data/Preprocessed_Data/Statement_Data.csv",parse_dates=True)

In [ ]:
# Statement=Statement.rename(columns={"Unnamed: 0":"index"})

# Exploratory Data Analysis

In [ ]:
Statement

In [ ]:
Statement.info()

In [ ]:
Statement.describe()

In [ ]:
Statement

In [ ]:
Statement['Amount']

In [ ]:
Statement=Statement.sort_index(ascending=False)
Cumulative_Balance=0
Cumulative_Balances=[]
types=list(Statement['Type'])
Amounts=list(Statement['Amount'])
for index in range(len(Amounts)):
    if types[index].strip()=="Dr":
        Cumulative_Balance-=round(Amounts[index],2)
    elif types[index].strip()=="Cr":
        Cumulative_Balance+=round(Amounts[index],2)
    Cumulative_Balances.append(Cumulative_Balance)
Statement['Cumulative_Balance']=Cumulative_Balances

In [ ]:
Statement

In [ ]:
Statement.sort_values(by="Date",ascending=True)

In [ ]:
Statement['Description'].value_counts()

In [ ]:
Statement['Type'].value_counts()

In [ ]:
Statement['Payment_Mode']=Statement['Description'].apply(getPaymentMethod)

In [ ]:
Statement['Recipient_Name']=Statement['Description'].apply(getRecipientName)

In [ ]:
Statement

In [ ]:
Statement['Payment_Mode'].value_counts()

In [ ]:
Statement[Statement['Payment_Mode']=="Automated Clearing House Payment"].sort_values(by="Date",ascending=False)

In [ ]:
Statement[Statement['Type']=="Cr"]

In [ ]:
Statement[Statement['Type']=="Dr"]

In [ ]:
Statement[Statement['Payment_Mode']=="Other"]

In [ ]:
Statement['Recipient_Name'].value_counts().head(20)

In [ ]:
Statement['Payment_Mode'].value_counts().head(20).plot.pie()

In [ ]:
Statement[Statement['Recipient_Name']=="Other"]

In [ ]:
Statement.plot(y="Cumulative_Balance")

In [ ]:
list(Statement['Cumulative_Balance'])

In [ ]:
list(Statement['Date'].to_numpy())

In [ ]:
list(range(len(Statement['Cumulative_Balance'])))